In [1]:
import pandas as pd
import os

In [3]:
def convert_date(date_str):
    try:
        if len(date_str) == 4 and date_str.isdigit():
            datetime_obj = pd.to_datetime('01/01/' + date_str, format='%m/%d/%Y')
        elif '-' in date_str and len(date_str.split('-')[1]) == 2:
            datetime_obj = pd.to_datetime(date_str, format='%b-%y')
        else:
            datetime_obj = pd.to_datetime(date_str, format='%m/%d/%y')
            if datetime_obj.year > 2053:
                datetime_obj = datetime_obj.replace(year=datetime_obj.year - 100)
        return datetime_obj.strftime('%Y-%m-%d')
    except:
        return None

In [9]:
AD = pd.read_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\raw\\annual_data.csv')
AD.dropna(axis=1, how='all', inplace=True)
AD['Data Series'] = AD['Data Series'].str.replace(',', '').str.replace(' ', '_')
reference_table = AD[['Data Series', 'Category', 'Source', 'Unit']]
reference_table['Description'] = None
reference_table.to_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\Annual_Data_reference_table.csv', index=False)
time_series_part = AD.set_index('Data Series').drop(columns=['SeriesID', 'Category', 'Source', 'Unit'])
time_series_AD = time_series_part.transpose().reset_index()
time_series_AD = time_series_AD.rename(columns={'index': 'date'})
time_series_AD['date'] = time_series_AD['date'].apply(convert_date)
time_series_AD.to_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\time_series\\annual_data.csv', index=False)

C:\Users\26871\AppData\Local\Temp\ipykernel_14080\368197076.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reference_table['Description'] = None


In [11]:
PA = pd.read_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\raw\\pair_analysis.csv')
PA.dropna(axis=1, how='all', inplace=True)
PA['Data Series'] = PA['Data Series'].str.replace(',', '').str.replace(' ', '_')
reference_table = PA[['Data Series', 'Category', 'Source', 'Unit']]
reference_table['Description'] = None
reference_table.to_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\Pair_Analysis_reference_table.csv', index=False)
time_series_part = PA.set_index('Data Series').drop(columns=['SeriesID', 'CatID', 'UOMID', 'Category', 'Source', 'Unit'])
time_series_PA = time_series_part.transpose().reset_index()
time_series_PA = time_series_PA.rename(columns={'index': 'date'})
time_series_PA['date'] = time_series_PA['date'].apply(convert_date)
time_series_PA.to_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\time_series\\pair_analysis.csv', index=False)

C:\Users\26871\AppData\Local\Temp\ipykernel_14080\4150825191.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reference_table['Description'] = None


In [5]:
def EI_transfer(file_name):
    df = pd.read_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\test\\' + file_name)
    first = df.iloc[:, :6]
    rest = df.iloc[:, 6:]
    rest.dropna(axis=1, how='all', inplace=True)
    df = pd.concat([first, rest], axis=1)
    
    first_column_name = df.columns[0]
    df_extracted = pd.DataFrame()
    df_extracted['Data Series'] = df[first_column_name]
    df_extracted = pd.concat([df_extracted['Data Series'], df[['Category', 'Source', 'Unit']]], axis=1)
    #df_extracted.to_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\EI_reference_table.csv', index=False, mode='a')
    
    df_melted = df.melt(id_vars=list(df.columns)[:6],
                    value_vars=list(df.columns)[6:],
                    var_name='date',
                    value_name='value')
    df_melted['date'] = df_melted['date'].apply(convert_date)
    df_melted.to_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\' + file_name, index=False)

In [6]:
directory = 'F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\test\\'
for filename in os.listdir(directory):
    if filename.startswith("EI_") and filename.endswith(".csv"):
        EI_transfer(filename)

In [14]:
def change_to_time_series(filename):
    df = pd.read_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\' + filename)

    earliest_date = pd.to_datetime(df['date']).min()
    dates = pd.date_range(start=earliest_date, end="2023-08-01", freq='D')
    output = pd.DataFrame({'date': dates})
    
    first_column_name = df.columns[0]
    
    df = df[df['Source'].str.upper() != 'FRED']
    
    time_series_df = df.pivot(index='date', columns=first_column_name, values='value').reset_index()
    
    source_map = df.set_index(first_column_name)['Source'].to_dict()
    time_series_df.columns = [f"{source_map[col]}_{col}" if col != 'date' else 'date' for col in time_series_df.columns]
    time_series_df['date'] = pd.to_datetime(time_series_df['date'])
    merged_df = output.merge(time_series_df, on='date', how='left')
    
    merged_df.dropna(subset=merged_df.columns[1:], how='all', inplace=True)
    merged_df.to_csv('F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\time_series\\' + filename, index=False)

In [15]:
directory = 'F:\\Python\\5900_capstone\\Metal-Price-Trend\\data\\'
for filename in os.listdir(directory):
    if filename.startswith("EI_") and filename.endswith(".csv"):
        change_to_time_series(filename)